In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import pandas as pd
import numpy as np
import pipit as pp

In [4]:
trace = pp.Trace.from_otf2("../../pipit/tests/data/ping-pong-otf2/")
pandas_df = trace.events[["Timestamp (ns)", "Name", "Event Type", "Process", "Thread"]].copy()
pandas_df["Process"] = pandas_df["Process"].astype(int)
# stream = pandas_df.groupby(["Process", "Thread"]).ngroup()
# sindex = pandas_df.groupby(['Process', 'Thread']).cumcount()
# pandas_df["idx"] = list(zip(stream, sindex))
# stream = pandas_df.groupby(["Process"]).ngroup()
idx = pandas_df.groupby(['Process']).cumcount()
pandas_df["idx"] = idx
pandas_df

,Timestamp (ns),Name,Event Type,Process,Thread,idx
0,0.000000e+00,ProgramBegin,Instant,1,0,0
1,3.008309e+04,"int main(int, char**)",Enter,1,0,1
2,4.028833e+04,MPI_Init,Enter,1,0,2
3,3.077309e+05,ProgramBegin,Instant,0,0,0
4,3.369797e+05,"int main(int, char**)",Enter,0,0,1
...,...,...,...,...,...,...
115,1.995748e+08,MPI_Finalize,Leave,1,0,57
116,1.995752e+08,"int main(int, char**)",Leave,0,0,58
117,1.995768e+08,"int main(int, char**)",Leave,1,0,58
118,1.996033e+08,ProgramEnd,Instant,0,0,59


In [15]:
pp.set_option("backend", "pandas")
pp.get_option("backend")

'pandas'

In [31]:
ds = pp.dsl.create_dataset()
ds.show()

:TraceDataset   (0 events)


In [32]:
# simulating a reader
row = pandas_df.iloc[0]
evt = pp.dsl.Event(
    rank=row["Process"],
    idx=row["idx"],
    thread=row["Thread"],
    timestamp=row["Timestamp (ns)"],
    event_type=row["Event Type"],
    name=row["Name"],
)
evt

:Event   {'rank': 1, 'idx': 0, 'thread': 0, 'timestamp': 0.0, 'event_type': 'Instant', 'name': 'ProgramBegin'}

In [33]:
ds.buffer

{}

In [34]:
ds.push_event(evt)
# ds.flush()
# ds.show()
ds.buffer

{1: [{'idx': 0,
   'thread': 0,
   'timestamp': 0.0,
   'event_type': 'Instant',
   'name': 'ProgramBegin'}]}

In [40]:
ds.flush()
ds.show()

+--------+-------+----------+-------------+--------------+--------------+
|   rank |   idx |   thread |   timestamp | event_type   | name         |
|--------+-------+----------+-------------+--------------+--------------|
|      1 |     0 |        0 |           0 | Instant      | ProgramBegin |
+--------+-------+----------+-------------+--------------+--------------+
:TraceDataset   (1 events)


In [29]:
ds.data[1]

,thread,timestamp,event_type,name
idx,,,,
0,0,0.0,Instant,ProgramBegin
0,0,0.0,Instant,ProgramBegin


In [22]:
ds.flush()
ds.show()

+--------+-------+----------+-------------+--------------+--------------+
|   rank |   idx |   thread |   timestamp | event_type   | name         |
|--------+-------+----------+-------------+--------------+--------------|
|      1 |     0 |        0 |           0 | Instant      | ProgramBegin |
|      1 |     0 |        0 |           0 | Instant      | ProgramBegin |
+--------+-------+----------+-------------+--------------+--------------+
:TraceDataset   (2 events)


In [10]:
for i in range(1, len(pandas_df)):
    row = pandas_df.iloc[i]
    evt = pp.dsl.Event(
        rank=row["Process"],
        idx=row["idx"],
        thread=row["Thread"],
        timestamp=row["Timestamp (ns)"],
        event_type=row["Event Type"],
        name=row["Name"],
    )
    ds.push_event(evt)

ds.flush()
ds.show()

+-------+--------+----------+--------------------+--------------+-----------------------+
| idx   | rank   | thread   | timestamp          | event_type   | name                  |
|-------+--------+----------+--------------------+--------------+-----------------------|
| 0     | 1      | 0        | 0.0                | Instant      | ProgramBegin          |
| 0     | 1      | 0        | 0.0                | Instant      | ProgramBegin          |
| 0     | 1      | 0        | 0.0                | Instant      | ProgramBegin          |
| 1     | 1      | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 2     | 1      | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| ...   | ...    | ...      | ...                | ...          | ...                   |
| 57    | 1      | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 58    | 0      | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 58    | 

In [11]:
ds.show()

+-------+--------+----------+--------------------+--------------+-----------------------+
| idx   | rank   | thread   | timestamp          | event_type   | name                  |
|-------+--------+----------+--------------------+--------------+-----------------------|
| 0     | 1      | 0        | 0.0                | Instant      | ProgramBegin          |
| 0     | 1      | 0        | 0.0                | Instant      | ProgramBegin          |
| 0     | 1      | 0        | 0.0                | Instant      | ProgramBegin          |
| 1     | 1      | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 2     | 1      | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| ...   | ...    | ...      | ...                | ...          | ...                   |
| 57    | 1      | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 58    | 0      | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 58    | 

In [165]:
ds.loc[1, 57]

:Event   {'rank': 1, 'idx': 57, 'thread': 0, 'timestamp': 199574793.63126454, 'event_type': 'Leave', 'name': 'MPI_Finalize'}

In [167]:
ds.loc[1, 50:57].show()

+--------+-------+----------+-------------+--------------+--------------+
|   rank |   idx |   thread |   timestamp | event_type   | name         |
|--------+-------+----------+-------------+--------------+--------------|
|      1 |    50 |        0 | 1.97951e+08 | Enter        | MPI_Recv     |
|      1 |    51 |        0 | 1.98502e+08 | Instant      | MpiRecv      |
|      1 |    52 |        0 | 1.98503e+08 | Leave        | MPI_Recv     |
|      1 |    53 |        0 | 1.98503e+08 | Enter        | MPI_Send     |
|      1 |    54 |        0 | 1.98504e+08 | Instant      | MpiSend      |
|      1 |    55 |        0 | 1.9932e+08  | Leave        | MPI_Send     |
|      1 |    56 |        0 | 1.9953e+08  | Enter        | MPI_Finalize |
|      1 |    57 |        0 | 1.99575e+08 | Leave        | MPI_Finalize |
+--------+-------+----------+-------------+--------------+--------------+
:TraceDataset   (8 events)


In [172]:
ds.loc[0:2, :].show()

+--------+-------+----------+--------------------+--------------+-----------------------+
| rank   | idx   | thread   | timestamp          | event_type   | name                  |
|--------+-------+----------+--------------------+--------------+-----------------------|
| 1      | 0     | 0        | 0.0                | Instant      | ProgramBegin          |
| 1      | 1     | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 1      | 2     | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| 0      | 0     | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| 0      | 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| ...    | ...   | ...      | ...                | ...          | ...                   |
| 1      | 57    | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 0      | 58    | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 1      |

In [174]:
ds.filter("name == 'MPI_Init'").show()

+--------+-------+----------+------------------+--------------+----------+
|   rank |   idx |   thread |        timestamp | event_type   | name     |
|--------+-------+----------+------------------+--------------+----------|
|      1 |     2 |        0 |  40288.3         | Enter        | MPI_Init |
|      0 |     2 |        0 | 346055           | Enter        | MPI_Init |
|      0 |     3 |        0 |      1.93643e+08 | Leave        | MPI_Init |
|      1 |     3 |        0 |      1.93644e+08 | Leave        | MPI_Init |
+--------+-------+----------+------------------+--------------+----------+
:TraceDataset   (4 events)


In [175]:
ds.filter("name in ('MPI_Init', 'MPI_Send', 'MPI_Recv')").show()

+--------+-------+----------+--------------------+--------------+----------+
| rank   | idx   | thread   | timestamp          | event_type   | name     |
|--------+-------+----------+--------------------+--------------+----------|
| 1      | 2     | 0        | 40288.33150186851  | Enter        | MPI_Init |
| 0      | 2     | 0        | 346054.77444467926 | Enter        | MPI_Init |
| 0      | 3     | 0        | 193643138.1741584  | Leave        | MPI_Init |
| 1      | 3     | 0        | 193643835.4832178  | Leave        | MPI_Init |
| 0      | 8     | 0        | 193668225.07270837 | Enter        | MPI_Send |
| ...    | ...   | ...      | ...                | ...          | ...      |
| 1      | 53    | 0        | 198503365.13620108 | Enter        | MPI_Send |
| 0      | 52    | 0        | 198506398.7408429  | Leave        | MPI_Send |
| 0      | 53    | 0        | 198506691.79201505 | Enter        | MPI_Recv |
| 1      | 55    | 0        | 199319910.70381415 | Leave        | MPI_Send |

In [151]:
ds.filter("timestamp > 1e5").show()

+--------+-------+----------+--------------------+--------------+-----------------------+
| rank   | idx   | thread   | timestamp          | event_type   | name                  |
|--------+-------+----------+--------------------+--------------+-----------------------|
| 0      | 0     | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| 0      | 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| 0      | 2     | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| 0      | 3     | 0        | 193643138.1741584  | Leave        | MPI_Init              |
| 1      | 3     | 0        | 193643835.4832178  | Leave        | MPI_Init              |
| ...    | ...   | ...      | ...                | ...          | ...                   |
| 1      | 57    | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 0      | 58    | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 1      |

In [152]:
ds.filter("name == 'MPI_Init'").show()

+--------+-------+----------+------------------+--------------+----------+
|   rank |   idx |   thread |        timestamp | event_type   | name     |
|--------+-------+----------+------------------+--------------+----------|
|      1 |     2 |        0 |  40288.3         | Enter        | MPI_Init |
|      0 |     2 |        0 | 346055           | Enter        | MPI_Init |
|      0 |     3 |        0 |      1.93643e+08 | Leave        | MPI_Init |
|      1 |     3 |        0 |      1.93644e+08 | Leave        | MPI_Init |
+--------+-------+----------+------------------+--------------+----------+
:TraceDataset   (4 events)


In [153]:
ds.filter("event_type == 'Enter'").show()

+--------+-------+----------+--------------------+--------------+-----------------------+
| rank   | idx   | thread   | timestamp          | event_type   | name                  |
|--------+-------+----------+--------------------+--------------+-----------------------|
| 1      | 1     | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 1      | 2     | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| 0      | 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| 0      | 2     | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| 0      | 4     | 0        | 193651646.20379105 | Enter        | MPI_Comm_size         |
| ...    | ...   | ...      | ...                | ...          | ...                   |
| 1      | 50    | 0        | 197951492.98251072 | Enter        | MPI_Recv              |
| 1      | 53    | 0        | 198503365.13620108 | Enter        | MPI_Send              |
| 0      |

In [163]:
ds.filter("name in ('MPI_Send', 'MPI_Recv')").show()

+--------+-------+----------+--------------------+--------------+----------+
| rank   | idx   | thread   | timestamp          | event_type   | name     |
|--------+-------+----------+--------------------+--------------+----------|
| 0      | 8     | 0        | 193668225.07270837 | Enter        | MPI_Send |
| 1      | 8     | 0        | 193677292.95417315 | Enter        | MPI_Recv |
| 0      | 10    | 0        | 193685930.32723847 | Leave        | MPI_Send |
| 0      | 11    | 0        | 193687379.3555098  | Enter        | MPI_Recv |
| 1      | 10    | 0        | 193696357.50795117 | Leave        | MPI_Recv |
| ...    | ...   | ...      | ...                | ...          | ...      |
| 1      | 53    | 0        | 198503365.13620108 | Enter        | MPI_Send |
| 0      | 52    | 0        | 198506398.7408429  | Leave        | MPI_Send |
| 0      | 53    | 0        | 198506691.79201505 | Enter        | MPI_Recv |
| 1      | 55    | 0        | 199319910.70381415 | Leave        | MPI_Send |

In [178]:
ds.data[0].eval("timestamp * 2")

idx
0     6.154619e+05
1     6.739595e+05
2     6.921095e+05
3     3.872863e+08
4     3.873033e+08
5     3.873063e+08
6     3.873113e+08
7     3.873136e+08
8     3.873365e+08
9     3.873452e+08
10    3.873719e+08
11    3.873748e+08
12    3.874314e+08
13    3.874377e+08
14    3.874877e+08
15    3.874888e+08
16    3.875291e+08
17    3.875297e+08
18    3.875685e+08
19    3.875690e+08
20    3.876511e+08
21    3.876519e+08
22    3.877099e+08
23    3.877104e+08
24    3.877793e+08
25    3.877800e+08
26    3.878841e+08
27    3.878850e+08
28    3.879919e+08
29    3.879923e+08
30    3.881000e+08
31    3.881009e+08
32    3.884106e+08
33    3.884114e+08
34    3.886056e+08
35    3.886063e+08
36    3.888175e+08
37    3.888184e+08
38    3.893508e+08
39    3.893523e+08
40    3.898230e+08
41    3.898235e+08
42    3.902633e+08
43    3.902644e+08
44    3.914360e+08
45    3.914377e+08
46    3.922796e+08
47    3.922801e+08
48    3.931679e+08
49    3.931689e+08
50    3.952265e+08
51    3.952283e+08
52    3.

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/02/23 07:23:58 WARN Utils: Your hostname, FEIG5S9893 resolves to a loopback address: 127.0.1.1; using 172.27.119.109 instead (on interface eth0)
24/02/23 07:23:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 07:24:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/23 07:24:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df = spark.createDataFrame(pandas_df)
df.show()

+--------------------+--------------------+----------+-------+------+
|      Timestamp (ns)|                Name|Event Type|Process|Thread|
+--------------------+--------------------+----------+-------+------+
|                 0.0|        ProgramBegin|   Instant|      1|     0|
|  30083.086937435106|int main(int, cha...|     Enter|      1|     0|
|   40288.33150186851|            MPI_Init|     Enter|      1|     0|
|   307730.9358165928|        ProgramBegin|   Instant|      0|     0|
|  336979.73374932166|int main(int, cha...|     Enter|      0|     0|
|  346054.77444467926|            MPI_Init|     Enter|      0|     0|
| 1.936431381741584E8|            MPI_Init|     Leave|      0|     0|
| 1.936438354832178E8|            MPI_Init|     Leave|      1|     0|
|1.9365164620379105E8|       MPI_Comm_size|     Enter|      0|     0|
|1.9365271483827707E8|       MPI_Comm_size|     Enter|      1|     0|
|1.9365316300611198E8|       MPI_Comm_size|     Leave|      0|     0|
|1.9365416291198432E

In [7]:
# # Filter rows where "Event Type" is "Enter"
# filtered_df = df.filter(col("Event Type") == "Enter")

# # Group by "Name" and "Process", then sum
# grouped_df = filtered_df.groupby(["Name", "Process"]).sum()

# # Group by "Name" and calculate the mean
# result_df = grouped_df.groupby("Name").mean()

# # Show the result
# result_df.show()

In [8]:
from pyspark.sql import functions as F

In [9]:
F.col("Name") > 5

Column<'(Name > 5)'>

In [10]:
# from pipit.dsl import functions as FF
# FF.col("Name")

In [11]:
import polars as pl

polars_df = pl.from_pandas(pandas_df)

polars_df.filter(pl.col("Process") == 0)

Timestamp (ns),Name,Event Type,Process,Thread
f64,cat,cat,i64,i64
307730.935817,"""ProgramBegin""","""Instant""",0,0
336979.733749,"""int main(int, …","""Enter""",0,0
346054.774445,"""MPI_Init""","""Enter""",0,0
1.9364e8,"""MPI_Init""","""Leave""",0,0
1.9365e8,"""MPI_Comm_size""","""Enter""",0,0
1.9365e8,"""MPI_Comm_size""","""Leave""",0,0
1.9366e8,"""MPI_Comm_rank""","""Enter""",0,0
1.9366e8,"""MPI_Comm_rank""","""Leave""",0,0
1.9367e8,"""MPI_Send""","""Enter""",0,0


In [12]:
F.col("Process")

Column<'Process'>

In [13]:
F.sum(F.col("Process") + F.col("Process"))

Column<'sum((Process + Process))'>

In [14]:
pl.col("Process") + pl.col("Process")

In [15]:
type(pl.col("Process"))

polars.expr.expr.Expr

In [3]:
!jt -t gruvboxl

In [167]:
pp.set_option("backend", "pandas")
pp.get_option("backend")

'pandas'

In [181]:
ds = pp.create_dataset(streams=[("process", "thread")])
ds.show()

:TraceDataset   [('process', 'thread')]   (0 events)


:Event {'index': (1, 0), 'process': 1, 'thread': 0, 'timestamp': 0.0, 'event_type': 'Instant', 'name': 'ProgramBegin'}

In [183]:
ds.push_event(evt)
ds.flush()
ds.show()

+---------+-----------+----------+-------------+--------------+--------------+
| index   |   process |   thread |   timestamp | event_type   | name         |
|---------+-----------+----------+-------------+--------------+--------------|
| (1, 0)  |         1 |        0 |           0 | Instant      | ProgramBegin |
+---------+-----------+----------+-------------+--------------+--------------+
:TraceDataset   [('process', 'thread')]   (1 events)


+---------+-----------+----------+--------------------+--------------+-----------------------+
| index   | process   | thread   | timestamp          | event_type   | name                  |
|---------+-----------+----------+--------------------+--------------+-----------------------|
| (1, 0)  | 1         | 0        | 0.0                | Instant      | ProgramBegin          |
| (1, 1)  | 1         | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| (1, 2)  | 1         | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| (0, 0)  | 0         | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| (0, 1)  | 0         | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| (0, 2)  | 0         | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| ...     | ...       | ...      | ...                | ...          | ...                   |
| (0, 57) | 0         | 0        | 199573648.15437

In [185]:
ds.filter("process == 0")

:TraceDataset   [('process', 'thread')]   (60 events)

In [186]:
ds.filter("process == 0").show()

+---------+-----------+----------+--------------------+--------------+-----------------------+
| index   | process   | thread   | timestamp          | event_type   | name                  |
|---------+-----------+----------+--------------------+--------------+-----------------------|
| (0, 0)  | 0         | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| (0, 1)  | 0         | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| (0, 2)  | 0         | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| (0, 3)  | 0         | 0        | 193643138.1741584  | Leave        | MPI_Init              |
| (0, 4)  | 0         | 0        | 193651646.20379105 | Enter        | MPI_Comm_size         |
| (0, 5)  | 0         | 0        | 193653163.00611198 | Leave        | MPI_Comm_size         |
| ...     | ...       | ...      | ...                | ...          | ...                   |
| (0, 54) | 0         | 0        | 199319973.70504

In [187]:
ds.filter("timestamp > 1e8").show()

+---------+-----------+----------+--------------------+--------------+-----------------------+
| index   | process   | thread   | timestamp          | event_type   | name                  |
|---------+-----------+----------+--------------------+--------------+-----------------------|
| (0, 3)  | 0         | 0        | 193643138.1741584  | Leave        | MPI_Init              |
| (1, 3)  | 1         | 0        | 193643835.4832178  | Leave        | MPI_Init              |
| (0, 4)  | 0         | 0        | 193651646.20379105 | Enter        | MPI_Comm_size         |
| (1, 4)  | 1         | 0        | 193652714.83827707 | Enter        | MPI_Comm_size         |
| (0, 5)  | 0         | 0        | 193653163.00611198 | Leave        | MPI_Comm_size         |
| (1, 5)  | 1         | 0        | 193654162.91198432 | Leave        | MPI_Comm_size         |
| ...     | ...       | ...      | ...                | ...          | ...                   |
| (0, 57) | 0         | 0        | 199573648.15437

In [188]:
ds.filter("event_type == 'Enter'").show()

+---------+-----------+----------+--------------------+--------------+-----------------------+
| index   | process   | thread   | timestamp          | event_type   | name                  |
|---------+-----------+----------+--------------------+--------------+-----------------------|
| (1, 1)  | 1         | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| (1, 2)  | 1         | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| (0, 1)  | 0         | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| (0, 2)  | 0         | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| (0, 4)  | 0         | 0        | 193651646.20379105 | Enter        | MPI_Comm_size         |
| (1, 4)  | 1         | 0        | 193652714.83827707 | Enter        | MPI_Comm_size         |
| ...     | ...       | ...      | ...                | ...          | ...                   |
| (0, 50) | 0         | 0        | 197613248.45135

In [189]:
ds.filter("process > thread").show()

+---------+-----------+----------+--------------------+--------------+-----------------------+
| index   | process   | thread   | timestamp          | event_type   | name                  |
|---------+-----------+----------+--------------------+--------------+-----------------------|
| (1, 0)  | 1         | 0        | 0.0                | Instant      | ProgramBegin          |
| (1, 1)  | 1         | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| (1, 2)  | 1         | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| (1, 3)  | 1         | 0        | 193643835.4832178  | Leave        | MPI_Init              |
| (1, 4)  | 1         | 0        | 193652714.83827707 | Enter        | MPI_Comm_size         |
| (1, 5)  | 1         | 0        | 193654162.91198432 | Leave        | MPI_Comm_size         |
| ...     | ...       | ...      | ...                | ...          | ...                   |
| (1, 54) | 1         | 0        | 198503650.55086

In [190]:
ds.streams

[('process', 'thread')]

In [206]:
ds.loc[1, 59]

:Event {'index': (1, 59), 'process': 1, 'thread': 0, 'timestamp': 199604459.57369962, 'event_type': 'Instant', 'name': 'ProgramEnd'}

In [207]:
ds.loc[:, 59]

IndexError: single positional indexer is out-of-bounds

In [ ]:
from pipit.dsl._pandas import PandasInterface
from pipit.dsl.event import Event
from pipit.dsl.expr import Column

ds = PandasInterface()

In [ ]:
ds.show()

In [ ]:
ds.push_event(evt)

In [ ]:
ds.flush()
ds.show()

In [ ]:
ds.flush()
ds.show()

In [ ]:
ds.filter("process == 0").show()